# 导入相关包

In [20]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime 
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"


def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

# a='train_output'
# b="train"+r"_\d"
# print(re.match(b,a))
def csv_2_df(file_path):
    file_list=file_path.parents[0].glob("*.csv")
    result=None
    for item in file_list:
        if re.match(file_path.stem.split("_")[0]+r"_\d",item.stem) is not None:
            print(item.name)
            result=pd.read_csv(item) if result is None else pd.concat([result,pd.read_csv(item)])
    return result

test_path = pl.Path('datasets/train_1.csv')

test_df = csv_2_df(test_path)
test_df.to_csv("datasets/train.csv",index=None)

# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [21]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = 'datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  'datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [22]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

# 数据处理
## 抽取整体数据（一个sampleid内此字段内容都相同）
## 公告时间，实际购买公司

#### 1.抽取公告时间

In [23]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断验证集的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 4438.02it/s]


0.4583333333333333

100%|██████████| 8660/8660 [00:01<00:00, 6869.16it/s]


#### 2.抽取实际购买公司

In [24]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        if '公司' in i:
            return i

val_gm = val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
test_gm = test_df.progress_apply(lambda row:get_gm(row['text']), axis=1)

100%|██████████| 8660/8660 [00:01<00:00, 6825.82it/s]


#### 3.清洗提取出来的tabel数据，主要是清洗掉有问题的列 

In [25]:
# 将table转换格式以及处理
def deal_tabel(row):
    row = eval(row)
    if len(row)==0:
        return []
    else:
        new_row = []
        for i in row:
            for d in i:
                new_temp = []
                for h in d:
                    # 这里处理空数据或者错误的数据
                    h = str(h).replace('None', '').replace('\n','').replace(' ', '')                    
                    if h=='':
                        continue
                    if h=='.':
                        continue
                    if h=='/':
                        continue
                    new_temp.append(h)
                new_row.append(new_temp)
        # 这里判断是否构成一个完整得认购数据(通过一个list进行判断)
        new_new_row = []
        for i in new_row:
            if len(i) == 0:
                continue
            elif len(i) <= 4:
                continue
            else:
                new_new_row.append(i)
        return new_new_row
train_df_tabel = train_df['tabel'].progress_apply(lambda row:deal_tabel(row))
val_df_tabel = val_df['tabel'].progress_apply(lambda row:deal_tabel(row))
test_df_tabel = test_df['tabel'].progress_apply(lambda row:deal_tabel(row))

100%|██████████| 8660/8660 [00:03<00:00, 2554.16it/s]


#### 4.抽取的是单独的数据包含
#### 起息日，到息日， 金额，认购日期，产品发行方，理财产品

In [26]:
# 直接提取时间
# 如果出现两个时间第一个就是起息日，第二个就是到期日
# 如果出现一个时间就是起息日
# 出现的第一个money就是最后的金额
# 从这里面抽取所有序列
# 这里认为有逗号出现的就是money

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

from src.time_extractor import TimeFinder
import datetime
def get_list_data(df):
    df = list(df)
    new_df = []
    for i in tqdm(df):
        temp_df = []
        for h in i:
            new_h = []
            for digital in h:
                if ',' in digital:
                    # 这里也是为了统一数据有些是用元，有些是用万元
                    try:
                        ttt = float(digital.replace(',', '').replace('万元', '').replace('人民币', '').replace('元', ''))
                    except Exception:
                        continue
                    if ttt > 20000:
                        ttt = ttt/10000
                    new_h.append(ttt)
                else:
                    continue
            if len(new_h) == 0:
                continue
            temp_single = {}
            a = '_'.join(h)
            # 抽取时间和money
            t = TimeFinder()
            time_all = t.find_time(a)
            if time_all == None:
                continue
            rgrq = time_all[0]
            cpqxr = time_all[0]
            if len(time_all) > 1:
                try:
                    cpdxr = time_all[1]
                    # 相减
                    d1 = datetime.datetime.strptime(cpqxr, '%Y-%m-%d')
                    d2 = datetime.datetime.strptime(cpdxr, '%Y-%m-%d')
                    d = d2 - d1
                    cpqx = str(d.days) + '天'
                except Exception:
                    cpdxr = np.nan
                    cpqx = np.nan
            else:
                cpdxr = np.nan
                cpqx = np.nan
                
            # 筛选出除开数字与包含时间的列
            # 末尾是
            last_two = ['公司', '银行', '信托', '证券',  '分行', '支行', '中心', '业部', '商行', '建行']
            mowei = np.nan
            selected_bank_and_works = []
            for l in h:
                new_l = list(str(l))
                new_l_test = ''.join(l[-2:])
                if new_l_test in last_two:
                    mowei = l
                    continue
                if '资金' in l:
                    continue
                if '收益' in l:
                    continue
                if '到期' in l:
                    continue
                if ',' in l:
                    continue
                if '.' in l:
                    continue
                if '/' in l:
                    continue
                if '年' in l:
                    continue
                if '-' in l:
                    continue
                if len(l) < 4:
                    continue
                if is_number(l):
                    continue
                selected_bank_and_works.append(l)
            if len(selected_bank_and_works) < 1:
                continue
            
            temp_single['认购日期'] = rgrq
            temp_single['产品起息日'] = cpqxr
            temp_single['产品到期日'] = cpdxr
            temp_single['产品期限'] = cpqx
            temp_single['认购金额(万元)'] = new_h[0]
            temp_single['产品发行方名称'] = mowei
            temp_single['理财产品名称'] = selected_bank_and_works[0]
            temp_df.append(temp_single)
        new_df.append(temp_df)
    return new_df

val_contain_date = get_list_data(val_df_tabel)
test_contain_data = get_list_data(test_df_tabel) 

100%|██████████| 8660/8660 [13:15<00:00, 10.88it/s]


#### 5.汇总整理数据

In [27]:
# 将前面提取到的数据整理成对应格式
sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(val_df['sample_id'])
gg = list(val_gm)
time = list(val_time)
for i, value in enumerate(sample_id):
    for j in val_contain_date[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
val_result = result

In [28]:
sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(test_df['sample_id'])
gg = list(test_gm)
time = list(test_time)
for i, value in enumerate(sample_id):
    for j in test_contain_data[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
test_result = result
test_result

,sample_id,认购日期,理财产品名称,产品发行方名称,认购金额(万元),产品起息日,产品到期日,产品期限,实际购买公司名称,公告日期
0,11190,2016-06-28,星河文化,建设银行,1000.0,2016-06-28,NaN,NaN,北京京西文化旅游股份有限公司,2016-11-23
1,11190,2016-07-01,西藏星河,包商银行,11500.0,2016-07-01,2016-12-26,178天,北京京西文化旅游股份有限公司,2016-11-23
2,11190,2016-07-28,持有期不定,工商银行,13700.0,2016-07-28,NaN,NaN,北京京西文化旅游股份有限公司,2016-11-23
3,11190,2016-08-03,西藏星河,南京银行,1000.0,2016-08-03,2016-11-12,101天,北京京西文化旅游股份有限公司,2016-11-23
4,11190,2016-10-10,北京京艺,交通银行,1000.0,2016-10-10,2016-11-07,28天,北京京西文化旅游股份有限公司,2016-11-23
...,...,...,...,...,...,...,...,...,...,...
24566,22332,2015-10-16,平安银行卓越计划滚动型保本人民币公司理财产品,NaN,3900.0,2015-10-16,NaN,NaN,赛轮金宇集团股份有限公司关于,2016-05-12
24567,22337,2019-03-01,中国工商银行法人“添利宝”净值型理财产品（TLB1801),中国工商银行,10000.0,2019-03-01,NaN,NaN,北京东方新星石化工程股份有限公司,2019-03-04
24568,22338,2019-01-07,结构性存款,招商银行股份有限公司长沙韶山路支行,12000.0,2019-01-07,2019-03-08,60天,三诺生物传感股份有限公司,2019-01-72
24569,22340,2017-01-19,中信理财之共赢保本步步高升B款人民币理财产品,中信银行,4100.0,2017-01-19,NaN,NaN,丝路视觉科技股份有限公司,2018-01-25


# 建模过程（预处理模型）

### 1.采用LSTM网络用提取好的部分跟pdf中的text做交互预测
#### 理财类型、资金来源、实际购买公司和上市公司关系、买卖方是否有关联关系

In [29]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
train_lstm_input = train_lstm_input.fillna('否')
# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
label_2 = LabelEncoder()
label_3 = LabelEncoder()
label_4 = LabelEncoder()

train_data = pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)
train_data['text_2'] = train_lstm_input['text'].astype(str)

train_data['label_1'] = label_1.fit_transform(train_lstm_input['理财类型'])
train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])


In [30]:
# 导入相关库
import os
import pandas as pd
from tqdm.autonotebook import *
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import accuracy_score
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import StratifiedKFold
from gensim.models import FastText, Word2Vec
import re
from keras.layers import *
from keras.models import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import *
from keras.layers.advanced_activations import LeakyReLU, PReLU
import keras.backend as K
from keras.optimizers import *
from keras.utils import to_categorical
import tensorflow as tf
import random as rn
import gc
import logging
import gensim
import jieba
tqdm.pandas()
os.environ['PYTHONHASHSEED'] = '0'
# 显卡使用（如没显卡需要注释掉）
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
np.random.seed(1024)
rn.seed(1024)
tf.random.set_seed(1024)

In [31]:
train_data['text_1'] = train_data['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
train_data['text_2'] = train_data['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
train_data.head(5)

0%|          | 0/27154 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\lsqlh\AppData\Local\Temp\jieba.cache
Loading model cost 0.862 seconds.
Prefix dict has been built successfully.
100%|██████████| 27154/27154 [12:03<00:00, 37.56it/s]


,text_1,text_2,label_1,label_2,label_3,label_4
0,中银 保本 理财 - 人民币 按期 开放 理财产品 _ 中国银行 股份 有限公司 广州 东山 支行,[ ' 证券 代码 ： 600728 证券 简称 ： 佳 都...,9,1,0,0
1,中银 保本 理财 - 人民币 按期 开放 理财产品 _ 中国银行 股份 有限公司 广州 东山 支行,[ ' 证券 代码 ： 600728 证券 简称 ： 佳 都...,9,1,0,0
2,与 利率 挂钩 的 结构性 产品 _ 中国民生银行 股份 有限公司,[ ' 证券 代码 ： 600211 ...,9,0,0,0
3,广发 银行 “ 薪 加薪 ” 16 号 XJXCKJ2578 _ 广发 银行 股份 有限公司...,[ ' 证券 代码 ： 002171 ...,9,1,2,0
4,兴业银行 “ 金 雪球 - 优悦 ” 保本 开放式 人民币 理财产品 ( 2M ) _ 兴业...,[ ' 证券 代码 ： 002171 ...,9,1,0,0


In [32]:
### Tokenizer 序列化文本
def set_tokenizer(docs, split_char=' ', max_len=100):
    '''
    输入
    docs:文本列表
    split_char:按什么字符切割
    max_len:截取的最大长度
    
    输出
    X:序列化后的数据
    word_index:文本和数字对应的索引
    '''
    tokenizer = Tokenizer(lower=False, char_level=False, split=split_char)
    tokenizer.fit_on_texts(docs)
    X = tokenizer.texts_to_sequences(docs)
    maxlen = max_len
    X = pad_sequences(X, maxlen=maxlen, value=0)
    word_index=tokenizer.word_index
    return X, word_index, tokenizer

### 做embedding 这里采用word2vec 可以换成其他例如（glove词向量）
def trian_save_word2vec(docs, embed_size=300, save_name='w2v.txt', split_char=' '):
    '''
    输入
    docs:输入的文本列表
    embed_size:embed长度
    save_name:保存的word2vec位置
    
    输出
    w2v:返回的模型
    '''
    input_docs = []
    for i in docs:
        input_docs.append(i.split(split_char))
    logging.basicConfig(
    format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)
    w2v = Word2Vec(input_docs, size=embed_size, sg=1, window=8, seed=1017, workers=24, min_count=1, iter=10)
    w2v.save(save_name)
    print("w2v model done")
    return w2v

# 得到embedding矩阵
def get_embedding_matrix(word_index, embed_size=300, Emed_path="w2v_300.txt"):
    embeddings_index = Word2Vec.load(Emed_path)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector  
    print("null cnt",count)
    return embedding_matrix

# 得到fasttext矩阵
def load_fasttext(word_index, path):  
    count=0
    null_list=[]
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(path, encoding='utf-8') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words =  len(word_index)+1
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= nb_words: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        else:
            null_list.append(word)
            count+=1
    print("null cnt:",count)
    return embedding_matrix

def get_embedding_matrix_txt(word_index,embed_size=200,Emed_path="w2v_300.txt"):
    embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(
        Emed_path, binary=False)
    nb_words = len(word_index)+1
    embedding_matrix = np.zeros((nb_words, embed_size))
    count = 0
    for word, i in tqdm(word_index.items()):
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings_index[word]
        except:
            embedding_vector = np.zeros(embed_size)
            count += 1
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print("null cnt",count)
    return embedding_matrix

### 2.训练得到word2vec

In [33]:
text_1_list = np.unique(train_data['text_1'])
text_3_list = np.unique(train_data['text_2'])

print('开始序列化')
x1, index_1, token_1 = set_tokenizer(train_data['text_1'], split_char=' ', max_len=30)
x3, index_3, token_3 = set_tokenizer(train_data['text_2'], split_char=' ', max_len=600)
print('序列化完成')
gc.collect()

trian_save_word2vec(text_1_list, save_name='models/w2v_300_1.txt', split_char=' ')
gc.collect()
trian_save_word2vec(text_3_list, save_name='models/w2v_300_3.txt', split_char=' ')
gc.collect()

# 得到emb矩阵
emb1 = get_embedding_matrix(index_1, Emed_path='models/w2v_300_1.txt')
emb3 = get_embedding_matrix(index_3, Emed_path='models/w2v_300_3.txt')
gc.collect()

开始序列化
序列化完成


0

74:INFO:worker thread finished; awaiting finish of 16 more threads
2020-08-30 16:05:32,294:INFO:worker thread finished; awaiting finish of 15 more threads
2020-08-30 16:05:32,302:INFO:worker thread finished; awaiting finish of 14 more threads
2020-08-30 16:05:32,310:INFO:worker thread finished; awaiting finish of 13 more threads
2020-08-30 16:05:32,320:INFO:worker thread finished; awaiting finish of 12 more threads
2020-08-30 16:05:32,324:INFO:worker thread finished; awaiting finish of 11 more threads
2020-08-30 16:05:32,332:INFO:worker thread finished; awaiting finish of 10 more threads
2020-08-30 16:05:32,336:INFO:worker thread finished; awaiting finish of 9 more threads
2020-08-30 16:05:32,337:INFO:worker thread finished; awaiting finish of 8 more threads
2020-08-30 16:05:32,345:INFO:worker thread finished; awaiting finish of 7 more threads
2020-08-30 16:05:32,349:INFO:worker thread finished; awaiting finish of 6 more threads
2020-08-30 16:05:32,352:INFO:worker thread finished; awai

0

10:44,589:INFO:EPOCH 8 - PROGRESS: at 12.53% examples, 258177 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:45,601:INFO:EPOCH 8 - PROGRESS: at 14.82% examples, 254832 words/s, in_qsize 47, out_qsize 1
2020-08-30 16:10:46,613:INFO:EPOCH 8 - PROGRESS: at 17.63% examples, 256824 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:47,623:INFO:EPOCH 8 - PROGRESS: at 20.51% examples, 260702 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:48,671:INFO:EPOCH 8 - PROGRESS: at 22.36% examples, 260051 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:49,675:INFO:EPOCH 8 - PROGRESS: at 24.98% examples, 260755 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:50,705:INFO:EPOCH 8 - PROGRESS: at 28.00% examples, 261374 words/s, in_qsize 44, out_qsize 3
2020-08-30 16:10:51,709:INFO:EPOCH 8 - PROGRESS: at 30.89% examples, 261739 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:52,737:INFO:EPOCH 8 - PROGRESS: at 33.29% examples, 261778 words/s, in_qsize 47, out_qsize 0
2020-08-30 16:10:53,769:

0

2020-08-30 16:12:46,664:INFO:loading Word2Vec object from models/w2v_300_1.txt
2020-08-30 16:12:46,855:INFO:loading wv recursively from models/w2v_300_1.txt.wv.* with mmap=None
2020-08-30 16:12:46,857:INFO:setting ignored attribute vectors_norm to None
2020-08-30 16:12:46,858:INFO:loading vocabulary recursively from models/w2v_300_1.txt.vocabulary.* with mmap=None
2020-08-30 16:12:46,860:INFO:loading trainables recursively from models/w2v_300_1.txt.trainables.* with mmap=None
2020-08-30 16:12:46,861:INFO:setting ignored attribute cum_table to None
2020-08-30 16:12:46,862:INFO:loaded models/w2v_300_1.txt
100%|██████████| 7189/7189 [00:00<00:00, 54108.35it/s]
2020-08-30 16:12:47,037:INFO:loading Word2Vec object from models/w2v_300_3.txt
2020-08-30 16:12:47,153:INFO:loading wv recursively from models/w2v_300_3.txt.wv.* with mmap=None
2020-08-30 16:12:47,155:INFO:loading vectors from models/w2v_300_3.txt.wv.vectors.npy with mmap=None
2020-08-30 16:12:47,202:INFO:setting ignored attribute v

31

# 构建抽取模型

#### 1.网络结构

In [34]:
from keras.initializers import *

def model_conv(emb1, emb3):
    '''
    注意这个inputs
    seq1、seq2分别是两个输入
    是否做emb可选可不选，
    这个就是我们之前训练已经得到的用于embedding的（embedding_matrix1， embedding_matrix2）
    '''
    K.clear_session()

    emb_layer_1 = Embedding(
        input_dim=emb1.shape[0],
        output_dim=emb1.shape[1],
        weights=[emb1],
        input_length=30,
        trainable=False
    )
    
    emb_layer_3 = Embedding(
        input_dim=emb3.shape[0],
        output_dim=emb3.shape[1],
        weights=[emb3],
        input_length=600,
        trainable=False
    )
    
    
    seq1 = Input(shape=(30,))
    seq3 = Input(shape=(600,))    
    
    x1 = emb_layer_1(seq1)
    x3 = emb_layer_3(seq3)
    
    sdrop=SpatialDropout1D(rate=0.2)

    x1 = sdrop(x1)
    x3 = sdrop(x3)
     
    x = Dropout(0.2)(Bidirectional(GRU(128, return_sequences=True))(x1))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    x = Dropout(0.2)(Bidirectional(GRU(128, return_sequences=True))(x3))
    semantic = TimeDistributed(Dense(100, activation="tanh"))(x)
    merged_3 = Lambda(lambda x: K.max(x, axis=1), output_shape=(100,))(semantic)
    
    
    x = Multiply()([merged_1, merged_3])
    
    x = Dropout(0.2)(Activation(activation="relu")(BatchNormalization()(Dense(1000)(x))))
    x = Activation(activation="relu")(BatchNormalization()(Dense(500)(x)))
    pred_1 = Dense(10, activation='softmax')(x)
    pred_2 = Dense(3, activation='softmax')(x)
    pred_3 = Dense(3, activation='softmax')(x)
    pred_4 = Dense(2, activation='softmax')(x)
    model = Model(inputs=[seq1, seq3], outputs=[pred_1, pred_2, pred_3, pred_4])
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=0.0001),metrics=["accuracy"])
    return model
gc.collect()

0

In [35]:
model = model_conv(emb1, emb3)
model.summary()
l1 = to_categorical(train_data['label_1'], 10)
l2 = to_categorical(train_data['label_2'], 3)
l3 = to_categorical(train_data['label_3'], 3)
l4 = to_categorical(train_data['label_4'], 2)
model.fit([x1, x3],[l1, l2, l3, l4], batch_size=256, epochs=8, verbose=1, shuffle=True)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 600)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 30, 300)      2157000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 600, 300)     8537700     input_2[0][0]                    
_______________________________________________________________________________________

In [36]:
#保存权重
model.save_weights('models/lstm_model.h5')

#### 2.保存结果

In [37]:
# 预测验证集
val_result_for_pred = pd.merge(val_result, val_df, on='sample_id', how='left')
val_result_for_pred['text_1'] = val_result_for_pred['理财产品名称'].astype(str) + '_' + val_result_for_pred['产品发行方名称'].astype(str)
val_result_for_pred['text_2'] = val_result_for_pred['text'].astype(str)

val_result_for_pred['text_1'] = val_result_for_pred['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
val_result_for_pred['text_2'] = val_result_for_pred['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))

x1 = token_1.texts_to_sequences(val_result_for_pred['text_1'])
x1 = pad_sequences(x1, maxlen=30, value=0)
x3 = token_3.texts_to_sequences(val_result_for_pred['text_2'])
x3 = pad_sequences(x3, maxlen=600, value=0)
pred_result = model.predict([x1, x3], batch_size=1024, verbose=1)
pred_1 = label_1.inverse_transform(np.argmax(pred_result[0], axis=1))
pred_2 = label_2.inverse_transform(np.argmax(pred_result[1], axis=1))
pred_3 = label_3.inverse_transform(np.argmax(pred_result[2], axis=1))
pred_4 = label_4.inverse_transform(np.argmax(pred_result[3], axis=1))


13/13 [==============================] - 98s 8s/step


In [38]:
val_result['理财类型'] = pred_1
val_result['资金来源'] = pred_2
val_result['实际购买公司和上市公司关系'] = pred_3
val_result['买卖方是否有关联关系'] = pred_4

#### 3.离线验证评估

In [39]:
# 计算线下f1
# R（召回率）=抽取正确的记录数量/（抽取正确的目标数量+漏抽取的记录数量）
# P（准确率）=抽取正确的记录数量/（抽取错误的记录数量+抽取正确的记录数量）
def get_F1(val_pred, val_true):
    val_pred = list(val_pred)
    val_true = list(val_true)
    curr = list(set(val_pred).intersection(set(val_true)))
    R = len(curr)/len(val_true)
    P = len(curr)/len(val_pred)
    return 2*P*R/(P+R)

r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
val_true = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['理财类型'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str) + r['资金来源'].astype(str) + r['实际购买公司名称'].astype(str) + r['实际购买公司和上市公司关系'].astype(str) + r['买卖方是否有关联关系'].astype(str) + r['公告日期'].astype(str)

r = val_result
val_pred = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['理财类型'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到期日'].astype(str) + r['产品期限'].astype(str) + r['资金来源'].astype(str) + r['实际购买公司名称'].astype(str) + r['实际购买公司和上市公司关系'].astype(str) + r['买卖方是否有关联关系'].astype(str) + r['公告日期'].astype(str)

score = get_F1(val_pred, val_true)
score


0.020044780893485447

In [45]:
val_true_file=pl.Path("results/val_true.csv")
val_pred_file=pl.Path("results/val_pred.csv")

tr_true_file=pl.Path("results/tr_true.csv")

val_result.copy().sort_values(by="sample_id").sort_index(axis=1).to_csv(val_pred_file,index=None)
pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left').sort_index(axis=1).sort_values(by="sample_id").to_csv(val_true_file,index=None)
pd.merge(train_df[['sample_id']], train_outputs, on='sample_id', how='left').sort_index(axis=1).sort_values(by="sample_id").to_csv(tr_true_file,index=None)

#### 4.最终输出结果

In [21]:
# 预测测试集
test_result_for_pred = pd.merge(test_result, test_df, on='sample_id', how='left')
test_result_for_pred['text_1'] = test_result_for_pred['理财产品名称'].astype(str) + '_' + test_result_for_pred['产品发行方名称'].astype(str)
test_result_for_pred['text_2'] = test_result_for_pred['text'].astype(str)

test_result_for_pred['text_1'] = test_result_for_pred['text_1'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))
test_result_for_pred['text_2'] = test_result_for_pred['text_2'].progress_apply(lambda row:' '.join(jieba.lcut(str(row))))

x1 = token_1.texts_to_sequences(test_result_for_pred['text_1'])
x1 = pad_sequences(x1, maxlen=30, value=0)
x3 = token_3.texts_to_sequences(test_result_for_pred['text_2'])
x3 = pad_sequences(x3, maxlen=600, value=0)
pred_result = model.predict([x1, x3], batch_size=1024, verbose=1)
pred_1 = label_1.inverse_transform(np.argmax(pred_result[0], axis=1))
pred_2 = label_2.inverse_transform(np.argmax(pred_result[1], axis=1))
pred_3 = label_3.inverse_transform(np.argmax(pred_result[2], axis=1))
pred_4 = label_4.inverse_transform(np.argmax(pred_result[3], axis=1))

test_result['理财类型'] = pred_1
test_result['资金来源'] = pred_2
test_result['实际购买公司和上市公司关系'] = pred_3
test_result['买卖方是否有关联关系'] = pred_4

24/24 [==============================] - 158s 7s/step


In [22]:
test_result.to_csv('results/re_lstm_base.csv', index=False)